In [121]:
import gensim
from gensim.models import Word2Vec,KeyedVectors
import json
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer

In [128]:
from sklearn.cluster import KMeans, 
from sklearn.metrics import silhouette_score

In [87]:
with open('mixed.json') as f:
    data = json.load(f)

In [88]:
df = pd.DataFrame(data['results'])

In [89]:
df["d"]=df["d"]+" "+df["t"]
df["d"]=df["d"].str.lower()

In [90]:
df["d"]

0      the best of films and movie trailers. pick you...
1      duration:     3:11    posted:     3 days ago  ...
2      all new movie trailers are here! don't miss th...
3      duration:     8:33    posted:     26-feb-2012 ...
4      duration:     1:31:46    posted:     1 day ago...
                             ...                        
193    3 months free - offer terms & conditions: this...
194    we have over 15 years worth of information, ex...
195    music, film, tv and political news coverage. m...
196    get up to the minute news and reviews for all ...
197    explore the fundamentals of music via ableton'...
Name: d, Length: 198, dtype: object

In [91]:
en_sw = stopwords.words('english')

def remove_stopwords(text):
    result = []
    for token in text:
        if token not in en_sw:
            result.append(token)
            
    return result

def remove_pun(text):
    
    tokenizer = RegexpTokenizer(r"[a-zA-Z]{2,}")
    lst=tokenizer.tokenize(' '.join(text))
    return lst

In [92]:
df["d"]=df["d"].apply(word_tokenize)
df["d"]=df["d"].apply(remove_stopwords)
df["d"]=df["d"].apply(remove_pun)

In [96]:
print(df["d"].head())
processed_data=df["d"]

0    [best, films, movie, trailers, pick, favorite,...
1    [duration, posted, days, ago, video, marvel, s...
2    [new, movie, trailers, miss, latest, movie, tr...
3    [duration, posted, feb, video, movie, movie, y...
4    [duration, posted, day, ago, video, tippu, hin...
Name: d, dtype: object


In [97]:
model_cbow = Word2Vec(sentences=processed_data, sg=0, min_count=10, workers=4, window =4, epochs = 30)

In [100]:
model_skipgram = Word2Vec(sentences=processed_data, sg=1, min_count=10, workers=4, window =12, epochs = 30)

In [101]:
import pickle

In [104]:
file = open('model_cbow', 'wb')
pickle.dump(model_cbow, file)
file.close()

In [105]:
file1 = open('model_skipgram', 'wb')
pickle.dump(model_skipgram, file1)
file1.close()

In [106]:
### vectorise documents

In [114]:
model_skipgram.vector_size

100

In [118]:
def get_vectors(data,model):
    features=[]
    
    for tokens in data:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                vectors.append(model.wv[token])
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features

In [119]:
## cbow approach

In [122]:
vectorized_docs = get_vectors(processed_data, model=model_cbow)

In [126]:
print(vectorized_docs[:3])

[array([ 5.29822446e-02,  1.48875177e-01, -7.56810829e-02,  4.17144746e-02,
       -1.30066453e-02, -2.69784540e-01,  1.67102739e-01,  3.89526159e-01,
       -2.77235866e-01, -2.84033120e-01, -1.48629246e-03, -2.87166238e-01,
       -8.06015059e-02,  1.28564924e-01,  1.75853938e-01, -6.88514933e-02,
        1.72277093e-01, -7.26048872e-02, -1.20247722e-01, -3.49585056e-01,
        1.61939636e-01,  6.68865517e-02,  1.04377896e-01, -1.47156313e-01,
        1.24349780e-02,  3.62766581e-03, -2.83292979e-01, -1.45530724e-03,
       -9.90276411e-02,  7.82375187e-02,  2.05466837e-01, -1.12006374e-01,
        1.87238291e-01, -4.32474285e-01, -2.30624620e-02,  2.34506816e-01,
        1.00552849e-01,  1.44753814e-01, -5.41964062e-02, -1.76867068e-01,
        1.89891141e-02, -1.75833225e-01, -9.37447592e-04,  1.77479520e-01,
        1.58446968e-01, -6.83079809e-02, -2.02225164e-01, -7.36763626e-02,
        1.34006485e-01,  1.50081456e-01,  7.36067668e-02, -1.15843616e-01,
       -6.60186931e-02, 

In [139]:
def train_model(k,X):
    km=KMeans(n_clusters=k).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette score: {silhouette_score(X, km.labels_):0.2f}")

In [140]:
for i in range(2,50,2):
    train_model(i,vectorized_docs)

For n_clusters = 2
Silhouette score: 0.44
For n_clusters = 4
Silhouette score: 0.48
For n_clusters = 6
Silhouette score: 0.39
For n_clusters = 8
Silhouette score: 0.37
For n_clusters = 10
Silhouette score: 0.36
For n_clusters = 12
Silhouette score: 0.35
For n_clusters = 14
Silhouette score: 0.34
For n_clusters = 16
Silhouette score: 0.33
For n_clusters = 18
Silhouette score: 0.31
For n_clusters = 20
Silhouette score: 0.32
For n_clusters = 22
Silhouette score: 0.30
For n_clusters = 24
Silhouette score: 0.30
For n_clusters = 26
Silhouette score: 0.30
For n_clusters = 28
Silhouette score: 0.29
For n_clusters = 30
Silhouette score: 0.29
For n_clusters = 32
Silhouette score: 0.29
For n_clusters = 34
Silhouette score: 0.31
For n_clusters = 36
Silhouette score: 0.28
For n_clusters = 38
Silhouette score: 0.30
For n_clusters = 40
Silhouette score: 0.29
For n_clusters = 42
Silhouette score: 0.28
For n_clusters = 44
Silhouette score: 0.29
For n_clusters = 46
Silhouette score: 0.28
For n_clusters 

In [141]:
#skipgram approach

In [142]:
vectorized_docs_ = get_vectors(processed_data, model=model_skipgram)

In [143]:
print(vectorized_docs_[:3])

[array([ 0.09512119,  0.06467394, -0.03815909, -0.01189819, -0.04195003,
       -0.24698551,  0.10046784,  0.31836042, -0.17928486, -0.2350083 ,
        0.01185402, -0.18447576, -0.09351388,  0.11234062,  0.11318602,
       -0.02970089,  0.14187568, -0.08623372, -0.0982926 , -0.30623233,
        0.16551793,  0.09239277,  0.06060841, -0.18286058,  0.01381753,
       -0.0370081 , -0.174573  ,  0.02844322, -0.04531585,  0.10923936,
        0.16102031, -0.03327648,  0.16600698, -0.3532261 , -0.02993268,
        0.18699138,  0.08418787,  0.1155924 , -0.05607249, -0.12203065,
        0.09863321, -0.16993819,  0.02957912,  0.12400094,  0.16681279,
       -0.00871993, -0.11312783, -0.07471308,  0.09964582,  0.13533677,
        0.0524238 , -0.06539598, -0.01653696, -0.1584261 , -0.00746024,
       -0.04708894,  0.07630965,  0.06009493, -0.06103357,  0.07496937,
        0.01941229, -0.0226482 ,  0.11186621,  0.09223464, -0.04949639,
        0.2635463 ,  0.07107346,  0.18000095, -0.25387502,  0.1

In [144]:
for i in range(2,50,2):
    train_model(i,vectorized_docs_)

For n_clusters = 2
Silhouette score: 0.54
For n_clusters = 4
Silhouette score: 0.49
For n_clusters = 6
Silhouette score: 0.45
For n_clusters = 8
Silhouette score: 0.42
For n_clusters = 10
Silhouette score: 0.42
For n_clusters = 12
Silhouette score: 0.40
For n_clusters = 14
Silhouette score: 0.40
For n_clusters = 16
Silhouette score: 0.39
For n_clusters = 18
Silhouette score: 0.41
For n_clusters = 20
Silhouette score: 0.38
For n_clusters = 22
Silhouette score: 0.40
For n_clusters = 24
Silhouette score: 0.37
For n_clusters = 26
Silhouette score: 0.37
For n_clusters = 28
Silhouette score: 0.39
For n_clusters = 30
Silhouette score: 0.38
For n_clusters = 32
Silhouette score: 0.38
For n_clusters = 34
Silhouette score: 0.37
For n_clusters = 36
Silhouette score: 0.37
For n_clusters = 38
Silhouette score: 0.37
For n_clusters = 40
Silhouette score: 0.39
For n_clusters = 42
Silhouette score: 0.36
For n_clusters = 44
Silhouette score: 0.38
For n_clusters = 46
Silhouette score: 0.36
For n_clusters 